In [1]:
import numpy as np 
import matplotlib.pyplot as plt

from src.inputs.visualize import visualize_input
from src.models import V1_model_1 as V1_model
from src.inputs.examples import neighboring_textures

ModuleNotFoundError: No module named 'src'

In [ ]:
# Instantiate and test the FullModel
seed = 42
model = V1_model(seed=seed, alpha_x=1.0, alpha_y=1.0)
A, C = neighboring_textures(22, 60, 2.0)

T = 12.0
dt = 0.001
X_gen, Y_gen, I = model.simulate(
    A, C, dt=dt, T=T, verbose=False, noisy=True, mode="wrap"
)

# Time points in seconds to plot
time_points = np.array([0.7, 0.9, 1.2, 1.4, 1.8])
steps = [int(t / dt) for t in time_points]

# average of input across columns for the orientation of the target bar
X_per_column = np.concatenate(
    [model.g_x(X_gen[:, :, :30, 6]).mean(axis=1), model.g_x(X_gen[:, :, 30:, 0]).mean(axis=1)], axis=1
)
# Plot input
fig, axes = plt.subplots(2, 1, figsize=(6, 5), constrained_layout=True, dpi=400)
visualize_input(A, C, verbose=False, axis=axes[0])
axes[0].set_title(r"Input Image $\hat{I}_{i \theta}$ to model")

# visualize_output(A, I.max(axis=-1), verbose=True)
# n_cols = A.shape[1]
# plt.xticks(
#     np.arange(0, n_cols, 1),
#     labels=[f"{i}" for i in range(1, n_cols + 1)]
# )
# # plt.tick_params(axis='x', length=0)  # hide tick marks
# plt.show()

# Plot model dynamics, see fig. 5.21 in "Understanding Vision" (Li Zhaoping, 2014)
x_axis = np.arange(X_per_column.shape[1])  # column indices
for t_idx, step in zip(time_points, steps):
    # Neural response: sum over orientation channels (axis=-1)
    response = X_per_column[step]
    axes[1].plot(x_axis, response, label=f"t = {t_idx:.1f}")

avg_response = X_per_column.mean(axis=0)
axes[1].plot(
    x_axis, avg_response, label="Temporal average", linewidth=3, linestyle="--", color="k"
)

axes[1].set_xlabel("Texture column number")
axes[1].set_ylabel("Model response " + r"$g_x(x)$")
axes[1].set_ylim(-0.1, 1.1)
axes[1].set_xlim(0, 59)
axes[1].legend(framealpha=0.8, loc="upper left")
axes[1].grid(True)

plt.show()

- too fast?
- not as smooth?